In [19]:
import pandas as pd
from collections import Counter
orgdata = pd.read_csv("test.csv")
orgdata.head()

,編號,案類,發生日期,發生時段,發生地點,備註,seg
0,6,強盜,1080124,02~04,台北市文山區景行里車前路29巷,NaN,"['景行', '里車', '前路', '文山', '台北市']"
1,14,強盜,1090404,02~04,台北市大安區光武里忠孝東路4段101巷(大安路1段31巷),NaN,"['大安', '區光', '武里', '東路', '忠孝', '台北市']"
2,19,強盜,1100205,02~04,台北市大同區歸綏街151~180號,NaN,"['區歸', '綏街', '台北市', '大同']"


In [20]:
subdata = orgdata[['發生時段', '發生地點', 'seg']]

In [21]:
output_series = subdata.groupby(['發生地點'])['seg'].apply(lambda x: ','.join(x)).reset_index()

In [22]:
output_series

,發生地點,seg
0,台北市大同區歸綏街151~180號,"['區歸', '綏街', '台北市', '大同']"
1,台北市大安區光武里忠孝東路4段101巷(大安路1段31巷),"['大安', '區光', '武里', '東路', '忠孝', '台北市']"
2,台北市文山區景行里車前路29巷,"['景行', '里車', '前路', '文山', '台北市']"


In [23]:
output_series['seg'] = output_series['seg'].str.replace("', '", " ")
output_series

,發生地點,seg
0,台北市大同區歸綏街151~180號,['區歸 綏街 台北市 大同']
1,台北市大安區光武里忠孝東路4段101巷(大安路1段31巷),['大安 區光 武里 東路 忠孝 台北市']
2,台北市文山區景行里車前路29巷,['景行 里車 前路 文山 台北市']


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

docs = output_series['seg'].tolist()
vec = CountVectorizer()
X = vec.fit_transform(docs)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
df

C:\Users\will_\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,前路,區光,區歸,台北市,大同,大安,忠孝,文山,景行,東路,武里,綏街,里車
0,0,0,1,1,1,0,0,0,0,0,0,1,0
1,0,1,0,1,0,1,1,0,0,1,1,0,0
2,1,0,0,1,0,0,0,1,1,0,0,0,1


In [25]:
coocc = df.T.dot(df)
coocc

,前路,區光,區歸,台北市,大同,大安,忠孝,文山,景行,東路,武里,綏街,里車
前路,1,0,0,1,0,0,0,1,1,0,0,0,1
區光,0,1,0,1,0,1,1,0,0,1,1,0,0
區歸,0,0,1,1,1,0,0,0,0,0,0,1,0
台北市,1,1,1,3,1,1,1,1,1,1,1,1,1
大同,0,0,1,1,1,0,0,0,0,0,0,1,0
大安,0,1,0,1,0,1,1,0,0,1,1,0,0
忠孝,0,1,0,1,0,1,1,0,0,1,1,0,0
文山,1,0,0,1,0,0,0,1,1,0,0,0,1
景行,1,0,0,1,0,0,0,1,1,0,0,0,1
東路,0,1,0,1,0,1,1,0,0,1,1,0,0


In [26]:
import plotly.express as px
fig = px.imshow(coocc)
fig.layout.height = 1000
fig.layout.width = 1000
fig.show()

In [27]:
result = coocc.values.flatten()
len(result)

169

In [28]:
df_describe = pd.DataFrame(result)
df_describe.describe()

,0
count,169.000000
mean,0.455621
std,0.534061
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,3.000000


In [29]:
px.histogram(df)

In [30]:
import numpy as np

filter_num_up = 60
filter_num_down = 120
getid = coocc.index
important = pd.DataFrame(columns = getid).T
important['relation'] = ""
dropid = []

for i in range(len(coocc)):
    result = coocc[(coocc.iloc[i] > filter_num_up) & (coocc.iloc[i] < filter_num_down)].index.tolist()
    if(len(result) > 0):
        important['relation'].loc[getid[i]] = coocc[(coocc.iloc[i] > filter_num_up) & (coocc.iloc[i] < filter_num_down)].index.tolist()
        print(important['relation'].loc[getid[i]])
    else:
        dropid.append(i)
        important = important.drop([getid[i]]) 

In [31]:
important

,relation


In [32]:
np.asarray(getid[dropid])

array(['前路', '區光', '區歸', '台北市', '大同', '大安', '忠孝', '文山', '景行', '東路', '武里',
       '綏街', '里車'], dtype=object)

In [33]:
subcoocc = coocc.drop(np.asarray(getid[dropid]))

In [34]:
subcoocc

,前路,區光,區歸,台北市,大同,大安,忠孝,文山,景行,東路,武里,綏街,里車


In [35]:
subcoocc.drop(subcoocc.columns[dropid], axis=1, inplace=True)
subcoocc

""


In [40]:
fig = px.imshow(subcoocc)
fig.layout.height = 1000
fig.layout.width = 1000
fig.show()

In [41]:
from IPython.core.display import HTML
display(HTML(important.to_html()))

,relation


In [42]:
px.histogram(coocc)